In [3]:
import sys
import os
import numpy as np
import pandas as pd
import cv2
#import tensorflow as tf
import glob
import pcl
import math
import pickle
import six
import parse_xml as pt

In [4]:
import dask.array as da

import traceback
def pv(var):
    (filename,line_number,function_name,text)=traceback.extract_stack()[-2]
    print('%s: %s'%(text[text.find('(')+1:-1],var))


In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import model_01_conv
import layers
import input_helpers

In [6]:
# reload everything

import imp 

imp.reload(input_helpers)
imp.reload(layers)
imp.reload(model_01_conv)

None

In [7]:
# Clear graph
try:
    with model_01_conv.tf.Session() as sess:
        with model_01_conv.tf.device("/cpu:0"):
            model_01_conv.tf.reset_default_graph()
    None
except:
    pass

In [8]:
resolution=0.1
scale=8
voxel_shape=(800, 800, 80)
x=(-40, 40)
y=(-40, 40)
z=(-4, 4)

In [9]:
path_prefix = "/home/paperspace/Desktop/converted/car/training/suburu_leading_front_left/*/"

points_glob = path_prefix+"points/*.pcd"
labels_glob = path_prefix+"labels/*.pickle"

model_name = "3dcnn_800x800x80_res0.1_sc86.ckpt"

model_path = "/home/paperspace/Desktop/didi_solution/cnn_3d/"+model_name

points_paths = glob.glob(points_glob)
points_path = points_paths[600]

config = model_01_conv.tf.ConfigProto(device_count = {'GPU': 0})
sess = model_01_conv.tf.Session

with model_01_conv.tf.device("/cpu:0"):
    coord, objectness, centers, y_pred = model_01_conv.test(model_path, points_path,
                          voxel_shape=(800, 800, 80),
                          resolution=0.1,
                          scale=8, 
                          x=(-40, 40),
                          y=(-40, 40),
                          z=(-4, 4)
                          #y_thresh=0.75
                         )

INFO:tensorflow:Restoring parameters from /home/paperspace/Desktop/didi_solution/cnn_3d/3dcnn_800x800x80_res0.1_sc86.ckpt
objectness.shape:  (1, 100, 100, 10, 2)
objectness.max && min:  357.44 -294.982
y_pred.shape:  (1, 100, 100, 10, 2)
y_pred.max && min:  1.0 0.0
centers:  [[ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 ..., 
 [ 0 99 99]
 [ 0 99 99]
 [ 0 99 99]]
centers.shape:  (99948, 3)
voxels_shape:  (?, 800, 800, 80, 1)


In [75]:
pv(coord.shape)
pv(centers.shape)
pv(y_pred.shape)
pv(objectness.shape)
print("-")

co = coord[0]
objc = objectness[0, :, :, :, 1]
yp = y_pred[0, :, :, :, 1]

pv(objc.shape)
pv(co.shape)
pv(yp.shape)

label_path = input_helpers.get_label_path_for_point_path(points_path) 
pc = input_helpers.load_pc_from_pcd(points_path)
pv(pc.shape)


coord.shape: (1, 100, 100, 10, 24)
centers.shape: (1, 3)
y_pred.shape: (1, 100, 100, 10, 2)
objectness.shape: (1, 100, 100, 10, 2)
-
objc.shape: (100, 100, 10)
co.shape: (100, 100, 10, 24)
yp.shape: (100, 100, 10)
pc.shape: (30988, 3)


In [76]:
print("cmin: ", np.min(coord) )
print("cmax: ", np.max(coord) )

print("omin: ", np.min(objectness) )
print("omax: ", np.max(objectness) )

print("ymin: ", np.min(y_pred) )
print("ymax: ", np.max(y_pred) )

cmin:  -617.52
cmax:  618.859
omin:  -294.982
omax:  357.44
ymin:  0.0
ymax:  1.0


In [87]:
yp = y_pred[0, :, :, :, 0]
pv(yp.shape)
index = np.where(yp >= .995) 
index = np.array(index)

centers = np.vstack((index[0], np.vstack((index[1], index[2])))).transpose()
pv(index.shape)
pv(co.shape)

centers = input_helpers.sphere2center(centers,
                                resolution=resolution,
                                scale=scale,
                                min_value=np.array([x[0], y[0], z[0]]))

pv(centers.shape)
#print(coord[0][16][85])

yp.shape: (100, 100, 10)
index.shape: (3, 1)
co.shape: (100, 100, 10, 24)
centers.shape: (1, 3)


In [88]:
c0 = centers

In [103]:
pv(co.shape)
pv(index.shape)
pv(index[:,:].shape)
pv(centers.shape)
pv(co[index].shape)
corners = co[index[0], index[1], index[2]].reshape(-1, 3)[:, np.newaxis] + centers
#corners = co[index].T + centers
pv(corners.shape)

co.shape: (100, 100, 10, 24)
index.shape: (3, 1)
index[:,:].shape: (3, 1)
centers.shape: (1, 3)
co[index].shape: (3, 1, 100, 10, 24)
corners.shape: (8, 1, 3)


In [107]:
pv(pc.shape)
places, rots, size = input_helpers.read_labels(label_path)
bbox = input_helpers.get_boxcorners(places, rots, size)
pv(bbox.shape)

pc.shape: (30988, 3)
bbox.shape: (1, 8, 3)


In [108]:
publish_pc2(pc, corners.reshape(-1, 3), bbox.reshape(-1, 3))

In [83]:
print(centers.shape)

(99947, 3)


In [84]:
corners = objectness[0, :, :, :, 0][index].reshape(-1, 3)[:, np.newaxis] + centers
#corners2 = objectness[0, :, :, :, 1][index].reshape(-1, 3)[:, np.newaxis] + centers


MemoryError: 

In [79]:
pv(corners.shape)

corners.shape: (24000, 1, 3)


In [ ]:
publish_pc2(pc,  corners.reshape(-1, 8, 3).reshape(-1, 3))

In [23]:
import rospy
node = rospy.init_node("pc2_publisher")

In [24]:
from sensor_msgs.msg import PointCloud2
pub = rospy.Publisher("/points_raw", PointCloud2, queue_size=1000000)
pub2 = rospy.Publisher("/points_raw1", PointCloud2, queue_size=1000000)
pub3 = rospy.Publisher("/points_raw2", PointCloud2, queue_size=1000000)

In [47]:
def publish_pc2(pcloud, obj=None, obj2=None):
    """Publisher of PointCloud data"""
    import rospy
    import sensor_msgs.point_cloud2 as pc2
    from sensor_msgs.msg import PointCloud2
    import std_msgs
    from visualization_msgs.msg import Marker
    from geometry_msgs.msg import Point, TransformStamped

    r = rospy.Rate(0.1)
    
    ts = rospy.Time.now()
    
    header = std_msgs.msg.Header()
    header.stamp = ts
    header.frame_id = "velodyne"
    points = pc2.create_cloud_xyz32(header, pcloud)
    pub.publish(points)

    if obj is not None:
        header = std_msgs.msg.Header()
        header.stamp = ts
        header.frame_id = "velodyne"
        points2 = pc2.create_cloud_xyz32(header, obj)
        pub2.publish(points2)
        

    if obj2 is not None:
        header = std_msgs.msg.Header()
        header.stamp = ts
        header.frame_id = "velodyne"
        points3 = pc2.create_cloud_xyz32(header, obj2)
        pub3.publish(points3)

    #while not rospy.is_shutdown():

        
    #if obj is not None:
    #r.sleep()

In [ ]:
import sklearn
sklearn.utils.shuffle

In [ ]:
points_glob = "/home/paperspace/Desktop/converted/car/training/*/*/points/*.pcd"
labels_glob = "/home/paperspace/Desktop/converted/car/training/*/*/labels/*.pickle"

def get_label_path_for_point_path(point_path):
    ppath_parts = point_path.split('/')
    ppath_parts[-1] = ppath_parts[-1].replace('pcd', 'pickle')
    ppath_parts[-2] = 'labels'
    return os.path.join("/", *ppath_parts)
  
def visualize_points(points_path):#, label_path):
    label_path = get_label_path_for_point_path(points_path)
    pc = input_helpers.load_pc_from_pcd(points_path)
    places, rots, size = input_helpers.read_labels(label_path)

    #if(np.square(places[0]))
    place_dist = np.sqrt(np.sum(np.square(places[0])))

    if places is None or len(places.shape) == 0:
        print("places is none for: %s, %s" % (label_path, points_path))
        #continue
        return
    
    if place_dist > 50:
        print("places is too far away (%s) for: %s, %s"%(place_dist, label_path, points_path))
        #return
        #continue
    else: 
        print(points_path, place_dist)
        return True
    
    return None
        
    #corners = input_helpers.get_boxcorners(places, rots, size)
    #publish_pc2(pc, corners.reshape(-1, 3))
    #r.sleep()
    #break


points_paths = sorted( glob.glob(points_glob) )

labels_paths = sorted( glob.glob(labels_glob) )

#print(points_paths[0])
#print(labels_paths)

#visualize_points(points_paths[20])#, labels_paths[20])

#points_path = points_paths[20]
#pc = pcl.load(points_path)

#z = np.array(pc.to_list())
#z = np.array([[0, 0, 0]])
r = rospy.Rate(0.5)
for points_path in points_paths:
    if visualize_points(points_path):
        break

In [ ]:
points_paths[-1000]

In [ ]:
rospy.is_shutdown()

In [ ]:
def pc2voxel(pc, resolution=0.50, x=(0, 90), y=(-50, 50), z=(-4.5, 5.5), car_x=(-2, 2), car_y=(-2, 2), car_z=(-3, 2)):
    """Convert PointCloud2 to Voxel"""

    # Filter out or own car :-()
    car_logic_x = np.logical_and(pc[:, 0] > car_x[0], pc[:, 0] < car_x[1])
    car_logic_y = np.logical_and(pc[:, 1] > car_y[0], pc[:, 1] < car_y[1])
    car_logic_z = np.logical_and(pc[:, 2] > car_z[0], pc[:, 2] < car_z[1])
    car_filter_xyz = np.logical_and(car_logic_x, np.logical_and(car_logic_y, car_logic_z))

    logic_x = np.logical_and(pc[:, 0] >= x[0], pc[:, 0] < x[1])
    logic_y = np.logical_and(pc[:, 1] >= y[0], pc[:, 1] < y[1])
    logic_z = np.logical_and(pc[:, 2] >= z[0], pc[:, 2] < z[1])
    logic_xyz = np.logical_and(logic_x, np.logical_and(logic_y, logic_z))

    pc = pc[:, :3][np.logical_and(np.logical_not(car_filter_xyz), logic_xyz)]
    pc = ((pc - np.array([x[0], y[0], z[0]])) / resolution).astype(np.int32)

    voxel = np.zeros((int((x[1] - x[0]) / resolution),
                      int((y[1] - y[0]) / resolution),
                      int(round((z[1] - z[0]) / resolution))
                      ))
    voxel[pc[:, 0], pc[:, 1], pc[:, 2]] = 1
    return voxel

def get_boxcorners(places, rots, size):
    """Create 8 corners of bounding box from bottom center."""
    corners = []
    for place, rot, sz in zip(places, rots, size):
        x, y, z = place
        h, w, l = sz
        if l > 10:
            continue

        corner = np.array([
            [x - l / 2.0, y - w / 2.0, z],
            [x + l / 2.0, y - w / 2.0, z],
            [x - l / 2.0, y + w / 2.0, z],
            [x - l / 2.0, y - w / 2.0, z + h],
            [x - l / 2.0, y + w / 2.0, z + h],
            [x + l / 2.0, y + w / 2.0, z],
            [x + l / 2.0, y - w / 2.0, z + h],
            [x + l / 2.0, y + w / 2.0, z + h],
        ])

        corner -= np.array([x, y, z])

        rot_matrix = np.array([
            [np.cos(rot), -np.sin(rot), 0],
            [np.sin(rot), np.cos(rot), 0],
            [0, 0, 1]
        ])

        a = np.dot(corner, rot_matrix.transpose())
        a += np.array([x, y, z])
        corners.append(a)
    return np.array(corners)


In [ ]:
def create_label(places, size, corners, resolution=0.50, x=(0, 90), y=(-50, 50), z=(-4.5, 5.5), scale=4, min_value=None):
    """Create training Labels"""

    if min_value is None:
        #min_value = [0.0, 0.0, 0.0]
        min_value = [x[0], y[0], z[0]]
    min_value = np.array(min_value)

    places = np.array(places)
    size = np.array(size)
    corners = np.array(corners)

    x_logical = np.logical_and((places[:, 0] < x[1]), (places[:, 0] >= x[0]))
    y_logical = np.logical_and((places[:, 1] < y[1]), (places[:, 1] >= y[0]))
    z_logical = np.logical_and((places[:, 2] + size[:, 0] / 2.0 < z[1]), (places[:, 2] + size[:, 0] / 2.0 >= z[0]))
    xyz_logical = np.logical_and(x_logical, np.logical_and(y_logical, z_logical))

    center = places.copy()
    #center[:, 2] = center[:, 2] + size[:, 0] / 2.0  # Move bottom to center
    sphere_center = ((center[xyz_logical] - min_value) / (resolution * scale)).astype(np.int32)

    train_corners = corners[xyz_logical].copy()
    anchor_center = sphere2center(sphere_center,
                                     resolution=resolution,
                                     scale=scale,
                                     min_value=min_value)

    #for index, (corner, center) in enumerate(zip(corners[xyz_logical], anchor_center)):
    #    train_corners[index] = corner - center

    return sphere_center, train_corners


def sphere2center(p_sphere, resolution=0.5, scale=4, min_value=None):
    """from sphere center to label center"""

    if min_value is None:
        min_value = [0.0, 0.0, 0.0]
    min_value = np.array(min_value)

    center = p_sphere * (resolution * scale) + min_value
    return center

def corner_to_train(corners, sphere_center, resolution=0.50, x=(0, 90), y=(-50, 50), z=(-4.5, 5.5), scale=4, min_value=None):
    """Convert corner to Training label for regression loss"""

    if min_value is None:
        #min_value = [0.0, 0.0, 0.0]
        min_value = [x[0], y[0], z[0]]
    min_value = np.array(min_value)

    x_logical = np.logical_and((corners[:, :, 0] < x[1]), (corners[:, :, 0] >= x[0]))
    y_logical = np.logical_and((corners[:, :, 1] < y[1]), (corners[:, :, 1] >= y[0]))
    z_logical = np.logical_and((corners[:, :, 2] < z[1]), (corners[:, :, 2] >= z[0]))
    xyz_logical = np.logical_and(x_logical, np.logical_and(y_logical, z_logical)).all(axis=1)

    train_corners = corners[xyz_logical].copy()
    sphere_center = sphere2center(sphere_center,
                                     resolution=resolution,
                                     scale=scale,
                                     min_value=min_value)

    #for index, (corner, center) in enumerate(zip(corners[xyz_logical], sphere_center)):
        #train_corners[index] = corner - center

    return train_corners


def corner_to_voxel(voxel_shape, corners, sphere_center, scale=4):
    """Create final regression label from corner"""
    # import pdb
    # pdb.set_trace()
    corner_voxel = np.zeros((int(voxel_shape[0] / scale),
                             int(voxel_shape[1] / scale),
                             int(voxel_shape[2] / scale), 24))
    corner_voxel[sphere_center[:, 0], sphere_center[:, 1], sphere_center[:, 2]] = corners
    return corner_voxel

def create_objectness_label(sphere_center, resolution=0.5, x=90, y=100, z=10, scale=4):
    """Create Objectness label"""
    obj_maps = np.zeros((int(x / (resolution * scale)),
                         int(y / (resolution * scale)),
                         int(np.round(z / (resolution * scale)))
                         ))
    obj_maps[sphere_center[:, 0], sphere_center[:, 1], sphere_center[:, 2]] = 1
    return obj_maps



In [ ]:
points_path = "/home/paperspace/Desktop/converted/car/training/bmw_sitting_still/bmw01/points/1.pcd"
label_path = get_label_path_for_point_path(points_path) 
pc = input_helpers.load_pc_from_pcd(points_path)
places, rots, size = input_helpers.read_labels(label_path)
points = pc

In [ ]:
voxel_shape=(600, 600, 40)
resolution=0.2
scale=1
x=(-60, 60)
y=(-60, 60)
z=(-4, 4)
car_x=(-2, 2)
car_y=(-2, 2)
car_z=(-3, 2)

In [ ]:
("%sx%sx%s_"%voxel_shape)+("res%s_" % resolution)+("sc%s"%scale)

In [ ]:
corners = get_boxcorners(places, rots, size)
print("corners: ", corners.shape, corners)


In [ ]:


center_sphere, corner_label = create_label(places, size, corners,
                                           resolution=resolution,
                                           x=x,
                                           y=y,
                                           z=z,
                                           scale=scale,
                                           min_value=[x[0], y[0], z[0]])

print("center_sphere: ", center_sphere)
print("corner_label: ", corner_label.shape, corner_label)

g_map = create_objectness_label(center_sphere,
                                            resolution=resolution,
                                            x=(x[1] - x[0]),
                                            y=(y[1] - y[0]),
                                            z=(z[1] - z[0]),
                                            scale=scale)

print('g_map: ', np.argwhere(g_map > 0))

g_cord = corner_label.reshape(corner_label.shape[0], -1)
print(g_cord.shape)
g_cord = corner_to_voxel(voxel_shape, g_cord, center_sphere, scale=scale)
print(g_cord.shape)



In [ ]:
gc = np.argwhere(g_cord > 0)
gm = np.argwhere(g_map > 0)

In [ ]:
#model_01_conv.layers.cnn_model.layer2.get_shape().as_list()

print(gm.)

In [ ]:
print(gc.shape)
print(gm.shape)
print(gm)
print(center_sphere)

In [ ]:
print(gc)

In [ ]:
def nice_voxel(voxel):
    v = voxel

    v[:, 0] = v[:, 0] - np.min(v[:, 0])
    v[:, 1] = v[:, 1] - np.min(v[:, 1])
    v[:, 2] = v[:, 2] - np.min(v[:, 2])

    v[:, 0] = v[:, 0] - np.mean(v[:, 0])
    v[:, 1] = v[:, 1] - np.mean(v[:, 1])
    v[:, 2] = v[:, 2] - np.mean(v[:, 2])
    return v

In [ ]:
corner_label.shape

In [ ]:
"""Publisher of PointCloud data"""
import rospy
import sensor_msgs.point_cloud2 as pc2
from sensor_msgs.msg import PointCloud2
import std_msgs
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point, TransformStamped
from sensor_msgs.msg import PointCloud2

#voxel[:, 0] = np.mean(voxel[:, 0])

#publish_pc2(v)


v = np.argwhere(pc2voxel(pc, x=x, y=y, z=z, resolution=resolution)>0)
corn = np.argwhere(pc2voxel(corners[0], x=x, y=y, z=z, resolution=resolution))

v=nice_voxel(v)
corn=nice_voxel(corn)


r = rospy.Rate(0.5)
for _ in range(2):
    header = std_msgs.msg.Header()
    header.stamp = rospy.Time.now()
    header.frame_id = "velodyne"
    points = pc2.create_cloud_xyz32(header, v)
    pub3.publish(points)
    
    header = std_msgs.msg.Header()
    header.stamp = rospy.Time.now()
    header.frame_id = "velodyne"
    points = pc2.create_cloud_xyz32(header, corn)
    pub2.publish(points)
    
    
    #r.sleep()
